# Analysis of Aiforia Dopaminergic Axons 

## Part 0 - Outline
This code handles the automatic processing of raw data from mouse brains analyzed using the Aiforia model “Dopaminergic Axon Detector”. The data are stored in a local folder on the computer, specified within the code. We typically start with Excel files. To automatically change the format of multiple files, please refer to the notebook Change_Name_Format_Input_Data.ipynb.
This notebook is organized into three sections:

**1) Define Functions for Part 2 & 3**

**2) Automatic Analysis of N Slides (with Names Containing _S1) Across N Brains**

In this section, we automate the analysis of all N S1 slides corresponding to N brains (one slide per brain) contained in the folder with raw data. The approach is as follows:

1) All N filenames containing _S1 are collected from the folder and stored in a list.

2) For each slide (one per brain), the following steps are performed:
    a) Data analysis steps are executed.
    b) The results are exported to an Excel file specific to that brain.
    
After each loop iteration, the results for the current brain are added to an overview table containing the combined results for all brains. After the final loop, this overview table is also exported to an Excel file.

Note: In this section, we do not generate detailed “transposed” tables with information for each Striatum part.

**3) Automatic Analysis of N Slides (with Names Containing _S1) After Determining the (Un)Injected Areas**

In this section, we extend the analysis by identifying which brain regions are on the injected versus non-injected side. The comparison between both sides follows the same analysis workflow as in Section 2. Additionally, in this section we generate more detailed **transposed tables** containing information for each Striatum part for easier analysis.

## Part 1 - Define the necessary functions

### Part 1.1 - Load all necessary Python packages

In [ ]:
import functools
import glob
import math
import os

import pandas as pd
from IPython.display import display

# Pandas display options
pd.options.display.float_format = '{:.2f}'.format

### Part 1.2 - Data Locations

**TO DO:**

Specify the following paths before running the analysis:

1) **Raw data format:** choose the file format of the raw data (e.g.: excel, csv).
2) **Some experimental parameters:** spacing between sections and section thickness.
3) **Raw data folder:** the folder containing the original data files exported from Aiforia.
4) **Results folders:** the folders where the Excel files with processed results will be saved.
5) **Region mapping files:** the location of the Excel files that specify which brain regions have to be replaced and were (un)injected.

Use the following format to define each path: <font color='darkred'>r'file_location'</font>

In [ ]:
# Specify what data format you want to use for your raw data: excel, csv or feather. Do this by uncommenting the data_format that you want.
# data_format = 'excel'
data_format = 'csv'
# data_format = 'feather'

# Specify the experimental parameters (section_thickness in micrometers!!) 
spacing=12
section_thickness = 40

# Specify folder locations
folder_raw_data = r'C:\Users\...\Raw_data_TH_Axon'
folder_output_results = r'C:\Users\...\Output_Results_TH_Axon'
folder_output_results_injected = r'C:\Users\...\Output_Results_Injected_TH_Axon'
file_brainregions_to_replace =  r'C:\Users\...\Brainregions_To_Replace_TH_Axon.xlsx'
file_brainregions_injected =  r'C:\Users\...\Brainregions_Hemisphere_TH_Axon.xlsx'

In [ ]:
# Create output folders if they did not exist yet
if not os.path.isdir(folder_output_results):
    os.mkdir(folder_output_results)
if not os.path.isdir(folder_output_results_injected):
    os.mkdir(folder_output_results_injected)

### Part 1.3 – Function to Load All Image Files for Analysis

In [ ]:
def load_all_file_locations_S1(folder_raw_data: str, data_format: str) -> list[str]:
    """
    Create a list of all file locations for S1 images in the specified folder. 
    It is assumed that only S1 images are present in the folder, so '_S1' does not need to be in the filename here.

    Parameters
    ----------
    folder_raw_data : str
        Path to the folder containing raw data files.
    data_format : str
        Format of the raw data files ('excel', 'csv', or 'feather').

    Returns
    -------
    all_raw_data_file_locations_S1: list[str]
        Sorted list of full file paths for S1 images.
    """
    
    # Determine the file pattern based on the data format
    if data_format == 'excel':
        pattern = "*.xlsx"
    elif data_format == 'csv':
        pattern = "*.csv"
    elif data_format == 'feather':
        pattern = "*.feather"
    else:
        raise ValueError(
            "Invalid data format specified. Please set 'data_format' to 'excel', 'csv', or 'feather'."
        )
    
    # Get all matching files and sort them
    all_raw_data_file_locations_S1 = glob.glob(os.path.join(folder_raw_data, pattern))
    all_raw_data_file_locations_S1.sort()
    
    # Print the locations
    print("The location of all raw data files:")
    for file_location in all_raw_data_file_locations_S1:
        print(f" - {file_location}")
    
    return all_raw_data_file_locations_S1


### Part 1.4 – Function to Load the Brain Region Correction File

In [ ]:
def load_data_brainregions_to_replace(file_brainregions_to_replace: str) -> pd.DataFrame:
    """
    Load and clean the file containing corrections for brain regions that need to be replaced for each image.

    Parameters
    ----------
    file_brainregions_to_replace : str
        Path to the Excel file with columns: 'Image', 'Brainregion_Wrong', 'Brainregion_Correct'.

    Returns
    -------
    pd.DataFrame
        Cleaned DataFrame with brain regions to replace for each image. 
        All brain region names are stripped of spaces and converted to uppercase.
    """
    import pandas as pd
    import os

    if not os.path.exists(file_brainregions_to_replace):
        raise FileNotFoundError(f"The specified file does not exist: {file_brainregions_to_replace}")

    # Load the relevant columns from the Excel file
    df = pd.read_excel(
        file_brainregions_to_replace,
        usecols=['Image', 'Brainregion_Wrong', 'Brainregion_Correct'],
        dtype={'Image': 'str', 'Brainregion_Wrong': 'str', 'Brainregion_Correct': 'str'}
    )

    # Clean the data in place
    df['Image'] = df['Image'].str.strip()
    df['Brainregion_Wrong'] = df['Brainregion_Wrong'].str.upper().str.strip()
    df['Brainregion_Correct'] = df['Brainregion_Correct'].str.upper().str.strip()

    print("The modified table of brain regions to replace for each image:")
    display(df)
    
    return df


### Part 1.5 – Function to Load the File Indicating Which Brain Regions Were Injected

In [ ]:
def load_data_brainregions_injected(file_brainregions_injected: str) -> pd.DataFrame:
    """
    Load and clean the file specifying which brain regions were on the injected side for each specific image.

    Parameters
    ----------
    file_brainregions_injected : str
        Path to the Excel file containing injected brain region information.

    Returns
    -------
    pd.DataFrame
        Cleaned DataFrame with columns:
        - 'Image': image identifier
        - 'Brainregion': uppercase, stripped brain region name
        - 'Region_Hemisphere': uppercase, stripped hemisphere (injected side)
    """
    if not os.path.exists(file_brainregions_injected):
        raise FileNotFoundError(f"The specified file does not exist: {file_brainregions_injected}")

    # Load relevant columns from the Excel file
    df = pd.read_excel(
        file_brainregions_injected,
        usecols=['Image', 'Brainregion', 'Hemisphere'],
        dtype={'Image': 'str', 'Brainregion': 'str', 'Hemisphere': 'str'}
    )

    # Clean the data in place
    df['Image'] = df['Image'].str.strip()
    df['Brainregion'] = df['Brainregion'].str.upper().str.strip()
    df['Region_Hemisphere'] = df['Hemisphere'].str.upper().str.strip()
    
    # Drop the original 'Hemisphere' column
    df.drop(columns=['Hemisphere'], inplace=True)

    print("The modified table of injected brain regions for each image:")
    display(df)
    
    return df

### Part 1.6 – Function to Load a DataFrame and Clean It

In [ ]:
def dataframe_cleaning(file_location: str, df_brainregions_to_replace: pd.DataFrame, data_format: str) -> pd.DataFrame:
    """
    Load a raw data file for a specific image and clean it using the brain regions replacement table.
    Outputs a cleaned dataframe with additional calculated values like 'Lightness' and 'Intensity'.

    Parameters
    ----------
    file_location : str
        Path to the raw data file for a single image.
    df_brainregions_to_replace : pd.DataFrame
        DataFrame specifying which brain regions need to be replaced for each image.
    data_format : str
        Format of the raw data file: 'excel', 'csv', or 'feather'.

    Returns
    -------
    pd.DataFrame
        Cleaned dataframe with additional calculated columns.
    """
    # ------------------------
    # Load the raw data
    # ------------------------
    columns = ['Image', 'Area/object name', 'Area (μm²)', 'B', 'G', 'R']
    
    dtypes = {
        'Image': 'str',
        'Area/object name': 'str',
        'Area (μm²)': 'float64',
        'B': 'float64',
        'G': 'float64',
        'R': 'float64'
    }
    
    if data_format == 'excel':
        df = pd.read_excel(file_location, usecols=columns, dtype=dtypes, keep_default_na=True)
    elif data_format == 'csv':
        df = pd.read_csv(file_location, sep='\t', usecols=columns, dtype=dtypes, keep_default_na=True)
    elif data_format == 'feather':
        df = pd.read_feather(file_location)
        df = df.astype(dtypes)  # Ensure consistent column types
    else:
        raise ValueError("Invalid data format. Choose 'excel', 'csv', or 'feather'.")


    # ------------------------
    # Get image name from file name
    # ------------------------
    image_name = os.path.splitext(os.path.basename(file_location))[0]
    print('The present image =', image_name)
    df['Image'] = image_name  # Make sure the image name across the whole first column is correct

    # ------------------------
    # Clean and standardize names
    # ------------------------
    # Capitalize to never make mistakes against capitalization
    df['Area/object name'] = df['Area/object name'].str.upper()

    # Brain tissue detector rows are not needed
    df = df[~df['Area/object name'].str.contains("BRAIN TISSUE DETECTOR")]

    # ------------------------
    # Replace incorrect brain regions
    # ------------------------
    # Create the dictionary of brain regions that should be replaced for this specific image
    df_image_replacements = df_brainregions_to_replace[df_brainregions_to_replace['Image'] == image_name]
    dict_replace = pd.Series(
        df_image_replacements.Brainregion_Correct.values,
        index=df_image_replacements.Brainregion_Wrong
    ).to_dict()
    print(f"The dictionary of brain regions to replace for {image_name} is:", dict_replace)

    # Replace the value in the rows that have an Area/object name that is in dict_replace
    df['Area/object name'] = df['Area/object name'].replace(dict_replace, regex=False)
    
    #Remove rows where 'Area/object name' was replaced with 'EMPTY'
    df = df[df['Area/object name'] != 'EMPTY']
    
    # ------------------------
    # Merge similar area names by removing numbers
    # ------------------------
    df['Area/object name merged'] = df['Area/object name'].str.replace(r'\d+$', '', regex=True).str.strip()

    # ------------------------
    # Calculate Lightness and Intensity
    # ------------------------
    # Convert RGB values to grey scale intensity
    # The higher the 'lightness' value, the lighter the image as white is RGB=(255,255,255) and black = RGB(0,0,0). 
    # We use a scale 0 to 100, but boils down to the same.
    df['Lightness'] = 0.299*df['R'] + 0.587*df['G'] + 0.114*df['B']  # Standard grayscale conversion
    df['Intensity'] = 100 - df['Lightness']  # Higher Intensity = darker/more intense

    print('The full cleaned data =')
    display(df)

    return df


### Part 1.7 – Function to Calculate All Statistics (Disregarding Injected/Non-Injected Sides)

In [ ]:
def all_calculations(df: pd.DataFrame, section_thickness: float, spacing: float, groupby_column: str = 'Area/object name merged') -> pd.DataFrame:
    """
    Calculate total area, weighted intensity, and volume for each Area/object name merged.

    Parameters
    ----------
    df : pd.DataFrame
        Cleaned dataframe for a single image containing at least:
        'Area/object name merged', 'Area (μm²)', 'Intensity'.
    groupby_column : str, optional
        Column to group by for calculations (default is 'Area/object name merged').
    section_thickness : float
        Thickness of each section in μm.
    spacing : float
        Spacing between sections in μm.

    Returns
    -------
    pd.DataFrame
        DataFrame with total area, weighted intensity, and calculated volume for each region.
    """
    import pandas as pd
    import functools

    # ------------------------
    # Count number of objects per region
    # ------------------------
    df_counts = df.value_counts(groupby_column, sort=True).reset_index(name='Counts')

    # ------------------------
    # Calculate total area per region
    # ------------------------
    df_total_area = (
        df.groupby(groupby_column)
          .sum(numeric_only=True)['Area (μm²)']
          .rename_axis(groupby_column)
          .reset_index(name='Total Region Area (μm²)')
    )

    # ------------------------
    # Calculate weighted intensity per region 
    # ------------------------
    df_weighted_intensity = (
        df.groupby(groupby_column)[['Intensity', 'Area (μm²)']]
          .apply(lambda x: (x['Intensity'] * x['Area (μm²)']).sum() / x['Area (μm²)'].sum())
          .rename_axis(groupby_column)
          .reset_index(name='Weighted Intensity')
    )

    # ------------------------
    # Merge all calculated results
    # ------------------------
    df_all_calcs = functools.reduce(
        lambda left, right: pd.merge(left, right, on=groupby_column, how='outer'),
        [df_counts, df_total_area, df_weighted_intensity]
    )

    # Convert units and calculate volume
    df_all_calcs['Total Region Area (mm²)'] = df_all_calcs['Total Region Area (μm²)'] / 1_000_000
    df_all_calcs['Total Region Volume (mm³)'] = df_all_calcs['Total Region Area (mm²)'] * (section_thickness / 1000) * spacing

    # Drop original μm² column and sort by region name
    df_all_calcs.drop(columns=['Total Region Area (μm²)'], inplace=True)
    df_all_calcs.sort_values(by=[groupby_column], ascending=False, inplace=True)

    print('Calculations for each Area/object name merged:')
    display(df_all_calcs)

    return df_all_calcs


### Part 1.8 – Function to Calculate All Statistics for Injected/Uninjected Hemispheres

In [ ]:
def all_calculations_injected(df: pd.DataFrame, section_thickness: float, spacing: float, groupby_column: str = 'Region_Hemisphere') -> pd.DataFrame:
    """
    Calculate total area, weighted intensity, and volume for each hemisphere region,
    distinguishing injected vs. uninjected sides.

    Parameters
    ----------
    df : pd.DataFrame
        Cleaned dataframe containing at least 'Region_Hemisphere', 'Area (μm²)', and 'Intensity'.
    groupby_column : str, optional
        Column to group by for calculations (default 'Region_Hemisphere').
    section_thickness : float
        Thickness of each section in μm (must be provided).
    spacing : float
        Spacing between sections in μm (must be provided).

    Returns
    -------
    pd.DataFrame
        DataFrame with total area, weighted intensity, and calculated volume per hemisphere.
    """
    import pandas as pd
    import functools

    # ------------------------
    # Count number of objects per hemisphere
    # ------------------------
    df_counts = df.value_counts(groupby_column, sort=True).reset_index(name='Counts')

    # ------------------------
    # Calculate total area per hemisphere
    # ------------------------
    df_total_area = (
        df.groupby(groupby_column)
          .sum(numeric_only=True)['Area (μm²)']
          .rename_axis(groupby_column)
          .reset_index(name='Total Region Area (μm²)')
    )

    # ------------------------
    # Calculate weighted intensity per hemisphere
    # ------------------------
    df_weighted_intensity = (
        df.groupby(groupby_column)[['Intensity', 'Area (μm²)']]
          .apply(lambda x: (x['Intensity'] * x['Area (μm²)']).sum() / x['Area (μm²)'].sum())
          .rename_axis(groupby_column)
          .reset_index(name='Weighted Intensity')
    )

    # ------------------------
    # Merge all calculated results
    # ------------------------
    df_all_calcs = functools.reduce(
        lambda left, right: pd.merge(left, right, on=groupby_column, how='outer'),
        [df_counts, df_total_area, df_weighted_intensity]
    )

    # Convert units and calculate volume
    df_all_calcs['Total Region Area (mm²)'] = df_all_calcs['Total Region Area (μm²)'] / 1_000_000
    df_all_calcs['Total Region Volume (mm³)'] = df_all_calcs['Total Region Area (mm²)'] * (section_thickness / 1000) * spacing

    # Sort by hemisphere name
    df_all_calcs.sort_values(by=[groupby_column], ascending=False, inplace=True)

    print('Calculations on injected/uninjected regions:')
    display(df_all_calcs)

    return df_all_calcs


### Part 1.9 – Transposed Summary of Striatum Regions and Hemispheres

In [ ]:
def all_calculations_transposed(df1: pd.DataFrame, df2: pd.DataFrame, filter_column: str = 'Region_Hemisphere') -> pd.DataFrame:
    """
    Output a transposed dataframe showing area and intensity for each brain section (Striatum X),
    as well as total area and weighted intensity for injected/uninjected hemispheres.

    Parameters
    ----------
    df1 : pd.DataFrame
        Detailed data per brain section (columns: 'Region_Hemisphere', 'Area/object name', 'Intensity', 'Area (μm²)').
    df2 : pd.DataFrame
        Summary data per hemisphere (columns: 'Region_Hemisphere', 'Weighted Intensity', 'Total Region Area (μm²)').
    filter_column : str, optional
        Column specifying hemisphere regions (default 'Region_Hemisphere').

    Returns
    -------
    pd.DataFrame
        Transposed dataframe with all info, including calculated Loss.
    """
    image_name = df1['Image'].iloc[0]
    dictionary_inj_uninj = {}

    for region in ['STRIATUM UNINJECTED', 'STRIATUM INJECTED']:
        inj_uninj = region.split()[-1]  # This is either UNINJECTED or INJECTED

        # Transpose detailed section data
        df_detail = df1[df1[filter_column] == region][['Area/object name', 'Intensity', 'Area (μm²)']].transpose()
        
        # Change columns to UNINJECTED1', 'UNINJECTED2' ...  or 'INJECTED1', 'INJECTED2' ... 
        df_detail.columns = [f"{inj_uninj}{i}" for i in range(1, len(df_detail.columns) + 1)]

        # Rename index
        df_detail.index = [f"{image_name} Area name", f"{image_name} Intensity", f"{image_name} Area (μm²)"]

        # Transpose summary hemisphere data
        df_summary = df2[df2[filter_column] == region][['Weighted Intensity', 'Total Region Area (μm²)']].transpose()

        # Change columns to 'TOTAL UNINJECTED' or 'TOTAL INJECTED'
        df_summary.columns = [f"TOTAL {inj_uninj}"]

        # Rename index
        df_summary.index = [f"{image_name} Intensity", f"{image_name} Area (μm²)"]

        # Concatenate detailed and summary horizontally
        df_concatenated = pd.concat([df_detail, df_summary], axis=1)
        print(f'Transposed {region} regions with all info:')
        display(df_concatenated)

        dictionary_inj_uninj[region] = df_concatenated

    # Concatenate both hemispheres
    df_combined = pd.concat(dictionary_inj_uninj.values(), axis=1)

    # Calculate Loss (%)
    df_combined['Loss'] = 100 * (1 - pd.to_numeric(df_combined['TOTAL INJECTED'], errors='coerce') /
                                    pd.to_numeric(df_combined['TOTAL UNINJECTED'], errors='coerce'))

    # Move Loss column to the front
    cols = ['Loss'] + [col for col in df_combined.columns if col != 'Loss']
    df_combined = df_combined[cols]

    print('Dataframe combining uninjected and injected info:')
    display(df_combined)

    return df_combined


## Part 2 - Automatic Analysis of all N S1 Slides of all N Brains 


In [ ]:
%%time
# Measure the execution time of this cell

# Load the file specifying brain regions to replace/delete for each image
df_brainregions_to_replace = load_data_brainregions_to_replace(file_brainregions_to_replace)

# Get all file names containing '_S1' 
all_raw_data_file_locations_S1 = load_all_file_locations_S1(folder_raw_data, data_format)

# Initialize dictionary to store all overview dataframes
dictionary_overview_dataframes = {}

# Loop over all S1 images in the raw_data folder
for count, file_location_S1 in enumerate(all_raw_data_file_locations_S1):

    print(f'\nAnalysis of {file_location_S1}')
    
    # Extract image name from file_path
    image_name_S1 = os.path.splitext(os.path.basename(file_location_S1))[0]
    
    # Clean the S1 data
    df_S1_final = dataframe_cleaning(file_location_S1, df_brainregions_to_replace, data_format)

    # Perform calculations
    df_S1_all_calcs = all_calculations(df_S1_final, section_thickness, spacing)

    # Prepare overview dataframes. For the overview excel file, only the df_S1_all_calcs dataframe is needed. 
    # We will make 1 overview excelfile with a few tabpages that we store in dictionary_overview_dataframes:
    # dictionary_overview_dataframes = {Total Intensity : df,  Total Region Area: df, .... }
    list_calculation_results = ['Weighted Intensity', 'Total Region Area (mm²)', 'Total Region Volume (mm³)']

    for calculation_result in list_calculation_results:
        df_calc = df_S1_all_calcs[['Area/object name merged', calculation_result]].copy()
        df_calc.rename(columns={calculation_result: image_name_S1}, inplace=True)

        if count == 0:
            # Initialize dictionary on first loop
            dictionary_overview_dataframes[calculation_result] = df_calc
        else:
            # Merge subsequent results
            dictionary_overview_dataframes[calculation_result] = dictionary_overview_dataframes[calculation_result].merge(
                df_calc, how='outer', on='Area/object name merged'
            )

    # Clean up memory for next iteration
    del df_S1_final, df_S1_all_calcs

# Export overview tables to Excel
output_file_name_overview = os.path.join(folder_output_results, 'Overview_TH_Axons_Results.xlsx')

with pd.ExcelWriter(output_file_name_overview) as writer:
    for calculation_result in list_calculation_results:
        sheet_name = calculation_result.replace('/', ' per ')
        print(f'Overview dataframe with all {sheet_name} for all brains')
        display(dictionary_overview_dataframes[calculation_result])
        dictionary_overview_dataframes[calculation_result].to_excel(
            writer, sheet_name=sheet_name, index=False, float_format="%.3f"
        )


## Part 3 – Automatic Analysis of all N S1 Slides of all N Brains (Including Injected and Uninjected Hemispheres)


In [ ]:
%%time
# Measure the execution time of this cell

# Load the file specifying brain regions to replace/delete for each image
df_brainregions_to_replace = load_data_brainregions_to_replace(file_brainregions_to_replace)

# Load the file specifying which brain regions belong to which hemisphere for each image
df_brainregions_injected = load_data_brainregions_injected(file_brainregions_injected)

# Get all file names containing '_S1'
all_raw_data_file_locations_S1 = load_all_file_locations_S1(folder_raw_data, data_format)

# Initialize dictionary to store all overview dataframes
dictionary_overview_dataframes_injected = {}

# Loop over all S1 images in the raw_data folder
for count, file_location_S1 in enumerate(all_raw_data_file_locations_S1):

    print(f'\nAnalysis of {file_location_S1}')

    # Extract image name from file_path
    image_name_S1 = os.path.splitext(os.path.basename(file_location_S1))[0]

    # Clean the S1 data
    df_S1_final = dataframe_cleaning(file_location_S1, df_brainregions_to_replace, data_format)

    # Match cleaned data with injected/uninjected hemisphere definitions
    df_injected_object = df_brainregions_injected.merge(
        df_S1_final,
        left_on=['Image', 'Brainregion'],
        right_on=['Image', 'Area/object name'],
        how='inner'
    ).drop(columns=['Brainregion'])

    # Perform calculations
    df_S1_all_calcs_injected = all_calculations_injected(df_injected_object, section_thickness, spacing)
    df_uninjected_injected_info = all_calculations_transposed(df_injected_object, df_S1_all_calcs_injected)

    # For the overview excel file, only the df_S1_all_calcs_injected and df_uninjected_injected_info dataframe is needed. 
    # We will make 1 overview excelfiles with a few tabpages that we store in dictionary_overview_dataframes_injected:
    # dictionary_overview_dataframes_injected = {Intensities and Areas : df,  .... }
    if count == 0:
        dictionary_overview_dataframes_injected['Intensities and Areas'] = df_uninjected_injected_info
    else:
        dictionary_overview_dataframes_injected['Intensities and Areas'] = pd.concat(
            [dictionary_overview_dataframes_injected['Intensities and Areas'], df_uninjected_injected_info]
        )

    # Prepare the dataframes that are needed for the overview excel file: choose the needed columns,
    # and rename the header of the column with the values to the image_name 
    list_calculation_results = ['Total Region Area (mm²)', 'Total Region Volume (mm³)']

    for calculation_result in list_calculation_results:
        df_calc = df_S1_all_calcs_injected[['Region_Hemisphere', calculation_result]].copy()
        df_calc.rename(columns={calculation_result: image_name_S1}, inplace=True)

        if count == 0:
            dictionary_overview_dataframes_injected[calculation_result] = df_calc
        else:
            dictionary_overview_dataframes_injected[calculation_result] = dictionary_overview_dataframes_injected[
                calculation_result
            ].merge(df_calc, how='outer', on='Region_Hemisphere')

    # Clean up memory for next iteration
    del df_S1_final, df_injected_object, df_S1_all_calcs_injected, df_uninjected_injected_info


# After the for loops, we print the final overview tables
# First we make sure that the first row is 'STRIATUM UNINJECTED' and the second row is 'STRIATUM INJECTED'
dictionary_overview_dataframes_injected['Total Region Area (mm²)'].sort_values(
    by='Region_Hemisphere', ascending=False, inplace=True
)
dictionary_overview_dataframes_injected['Total Region Volume (mm³)'].sort_values(
    by='Region_Hemisphere', ascending=False, inplace=True
)

# Reorder the columns in the 'Intensities and Areas' overview
cols = dictionary_overview_dataframes_injected['Intensities and Areas'].columns
cols_loss = ['Loss']
# Sort based on the number in it, and make sure that the order is U1, U2 ... U9, U10... instead of U1, U10, U2...
cols_uninj = sorted([x for x in cols if 'UNINJECTED' in x and 'TOTAL' not in x],
                    key=lambda x: float(x.replace('UNINJECTED', '').strip()))
cols_total_uninj = ['TOTAL UNINJECTED']

cols_inj = sorted([x for x in cols if 'INJECTED' in x and 'UN' not in x and 'TOTAL' not in x],
                  key=lambda x: float(x.replace('INJECTED', '').strip()))
cols_total_inj = ['TOTAL INJECTED']

columns_ordered = cols_loss + cols_uninj + cols_total_uninj + cols_inj + cols_total_inj

dictionary_overview_dataframes_injected['Intensities and Areas'] = \
    dictionary_overview_dataframes_injected['Intensities and Areas'][columns_ordered]

# Extract loss values only for intensity rows
df = dictionary_overview_dataframes_injected['Intensities and Areas']
dictionary_overview_dataframes_injected['Intensity Loss'] = df.loc[df.index.str.contains('Intensity'), ['Loss']]

# Export results to Excel
output_file_name_overview = os.path.join(folder_output_results_injected, 'Overview_TH_Axons_Hemisphere_Results.xlsx')

list_calculation_results = [
    'Intensity Loss',
    'Intensities and Areas',
    'Total Region Area (mm²)',
    'Total Region Volume (mm³)'
]

with pd.ExcelWriter(output_file_name_overview) as writer:
    for calculation_result in list_calculation_results:
        calculation_result_clean = calculation_result.replace('/', ' per ')
        print(f'Overview dataframe: {calculation_result_clean}')
        display(dictionary_overview_dataframes_injected[calculation_result])
        dictionary_overview_dataframes_injected[calculation_result].to_excel(
            writer,
            sheet_name=calculation_result_clean,
            index=calculation_result in ('Intensity Loss', 'Intensities and Areas'),
            float_format="%.3f"
        )
